In [ ]:
MONK_TASK = 3
USER = 'Nunzio'

In [ ]:
import sys
import os
if (colab := 'google.colab' in sys.modules):
    from google.colab import drive
    drive.mount('/content/drive')
    BASE_PATH = '/content/drive/Shareddrives/Project_ML_23/' + USER + '/machine-learning-project'
    sys.path.insert(0,BASE_PATH)
    N_JOBS = 1
    sys.path.insert(0,BASE_PATH + '/src/utils')
    !pip install optuna
    !pip install scikit-learn
    !pip install scikeras
    TRAIN_DATA = os.path.join(BASE_PATH, 'datasets', 'monk',f'monks-{MONK_TASK}.train')
    TEST_DATA = os.path.join(BASE_PATH, 'datasets', 'monk',f'monks-{MONK_TASK}.test')
    IMAGES_FOLDER = os.path.join(BASE_PATH, 'images', 'monk',  f'task-{MONK_TASK}', 'neural_network')
    MODEL_FOLDER = os.path.join(BASE_PATH, 'trained_models', 'monk', f'task-{MONK_TASK}')
else :
    TRAIN_DATA = os.path.join('..', '..', '..', 'datasets', 'monk', f'monks-{MONK_TASK}.train')
    TEST_DATA = os.path.join('..', '..', '..', 'datasets', 'monk', f'monks-{MONK_TASK}.test')
    N_JOBS = -1
    IMAGES_FOLDER = os.path.join('..', '..', '..', 'images', 'monk', f'task-{MONK_TASK}', 'neural_network')
    MODEL_FOLDER = os.path.join('..', '..', '..', 'trained_models', 'monk', f'task-{MONK_TASK}')

In [ ]:
if (colab := 'google.colab' in sys.modules):
    sys.path.append(BASE_PATH + '/src/utils')
else:
    sys.path.append('../../utils')

import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns; sns.set_theme(style='darkgrid')

from utils import save_plot, set_random_state
from NN import MonkNeuralNetwork

set_random_state(42)

In [ ]:
# To skip the first column (row indexes)
columns_to_read = list(range(1, 8))

df_train = pd.read_csv(TRAIN_DATA, header=None, usecols=columns_to_read, delimiter=' ')
df_test = pd.read_csv(TEST_DATA, header=None, usecols=columns_to_read, delimiter=' ')
df_train.head()

In [ ]:
features = ['feature_' + str(i) for i in range(1, 7)]

# Rename columns
new_column_names = ['class'] + features

df_train.columns = new_column_names
df_test.columns = new_column_names

df_train.head()

In [ ]:
df_train_encoded = pd.get_dummies(df_train, columns=features)
df_test_encoded = pd.get_dummies(df_test, columns=features)

df_train_encoded, df_test_encoded = df_train_encoded.align(df_test_encoded, join='inner', axis=1)

df_train_encoded.head()

In [ ]:
features = df_train_encoded.columns.difference(['class'])

X_train = df_train_encoded[features].to_numpy()
y_train = df_train_encoded['class'].to_numpy()

X_test = df_test_encoded[features].to_numpy()
y_test = df_test_encoded['class'].to_numpy()

# Create model

In [ ]:
model = MonkNeuralNetwork(input_dim=X_train.shape[1], verbose=0)

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'architecture': [
        (8,), (8, 8), (8, 8, 8), (8, 8, 8, 8)
    ],
    'optimizer': ['sgd'],
    'activation': ['relu'],
    'learning_rate': [0.1, 0.3, 0.5],
    'momentum': [0.6, 0.8, 0.9],
    'lambda_value': [0.01],
    'epochs': [200],
    'batch_size': [8],
    'patience': [6]
}
grid_search = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    cv=5,
    n_jobs=N_JOBS,
    verbose=0,
    scoring='accuracy'
)

grid_search.fit(X_train, y_train)

In [ ]:
results = grid_search.cv_results_

results_df = pd.DataFrame(results)
csv_filename = f'Monk_NN_{MONK_TASK}.csv'
# results_df.to_csv(csv_filename, index=False)

# Learning curve

In [ ]:
from sklearn.model_selection import KFold

cv = KFold(n_splits=5, shuffle=True, random_state=42)
train_scores, test_scores = [], []

best_params = grid_search.best_params_

# Cross-validation with the best parameters
for train_index, test_index in cv.split(X_train):
    X_train_1, X_test_1 = X_train[train_index], X_train[test_index]
    y_train_1, y_test_1 = y_train[train_index], y_train[test_index]

    best_model = MonkNeuralNetwork(input_dim=X_train_1.shape[1], verbose=0, **best_params)
    best_model.fit(X_train_1, y_train_1, validation_data=(X_test_1, y_test_1))

    #df = best_model.history.history
    train_scores.append(best_model.history.history)

In [ ]:
train_scores_df = pd.DataFrame(train_scores)
train_scores_filename = f'Monk_NN_{MONK_TASK}_train_scores.csv'
# train_scores_df.to_csv(train_scores_filename, index=False)

train_scores_df['val_loss']

In [ ]:
loss, val_loss, acc, val_acc = [], [], [], []

for i in range(len(train_scores_df)):
    loss.append(train_scores_df.iloc[i]['loss'])
    val_loss.append(train_scores_df.iloc[i]['val_loss'])
    acc.append(train_scores_df.iloc[i]['binary_accuracy'])
    val_acc.append(train_scores_df.iloc[i]['val_binary_accuracy'])

df_loss = pd.DataFrame(loss).T
df_val_loss = pd.DataFrame(val_loss).T
df_acc = pd.DataFrame(acc).T
df_val_acc = pd.DataFrame(val_acc).T

mean_loss = df_loss.mean(axis=1)
var_loss = df_loss.var(axis=1)
mean_val_loss = df_val_loss.mean(axis=1)
var_val_loss = df_val_loss.var(axis=1)

mean_acc = df_acc.mean(axis=1)
var_acc = df_acc.var(axis=1)
mean_val_acc = df_val_acc.mean(axis=1)
var_val_acc = df_val_acc.var(axis=1)

plt.figure(figsize=(12, 5))

# loss and val_loss
plt.subplot(1, 2, 1)
plt.plot(mean_loss, label='Loss', linestyle='-')
plt.fill_between(range(len(mean_loss)), mean_loss-var_loss, mean_loss+var_loss, alpha=0.2)
plt.plot(mean_val_loss, label='Val Loss', linestyle='--')
plt.fill_between(range(len(mean_val_loss)), mean_val_loss-var_val_loss, mean_val_loss+var_val_loss, alpha=0.2)
plt.title('Learning Curve for Loss (MSE)')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

# binary_accuracy and val_binary_accuracy
plt.subplot(1, 2, 2)
plt.plot(mean_acc, label='Accuracy', linestyle='-')
plt.fill_between(range(len(mean_acc)), mean_acc-var_acc, mean_acc+var_acc, alpha=0.2)
plt.plot(mean_val_acc, label='Val Accuracy', linestyle='--')
plt.fill_between(range(len(mean_val_acc)), mean_val_acc-var_val_acc, mean_val_acc+var_val_acc, alpha=0.2)
plt.title('Learning Curve for Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
save_plot(plt, IMAGES_FOLDER, 'learning_curve')
plt.show()

# Test model

In [ ]:
from sklearn.metrics import accuracy_score, classification_report

# Test set accuracy hold-out

final_model = grid_search.best_estimator_
y_pred = final_model.predict(X_test)

print('Best parameters: ', grid_search.best_params_)
print('Best accuracy: ', grid_search.best_score_)
print('Test set accuracy: ', accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

# Save model

In [ ]:
from joblib import dump

model_path = os.path.join(MODEL_FOLDER, 'NN_model.joblib')
dump(final_model, model_path, compress=3)